In [4]:

def is_square_free( f ) :
    """
    This function checks if the polynomial `f` is square-free.
    """
    return gcd( f, f.derivative() ) == 1
    
def rad( f ) :
    """
    This function computes the radical of `f`.
    """
    # find the ring of polynomials
    P = f.parent()
    # return the radical
    return P(f/gcd(f,f.derivative()))
    
def square_free_decomposition( f, omit_trivial_factors = true ) :
    """
    This function takes a polynomial f and returns its square-free 
    decomposition computed using Tobey-Horowitz algorithm.
    The additional parameter omit_trivial_factors constrols whether factors = 1
    should be omitted (default) or not.
    """
    # The base ring must be a field of characteristic zero
    assert f.base_ring().characteristic() == 0, "The characteristic must be zero!"
    assert f.base_ring().is_field(), "The base ring must be a field!"

    # Store the ring of polys in P
    P = f.parent()
    # normalize f
    lcf = f.leading_coefficient()
    f = f.monic()
    # compute a0, a1 and b1
    a0 = f
    a1 = gcd(a0, a0.derivative())
    b1 = P( a0/a1 )
    # the list of square-free factors
    G = []
    # the counter
    j = 1
    # main loop
    while b1 != 1 :
        # compute next aj
        a0 = a1
        a1 = gcd(a0, a0.derivative()) 
        # compute next bj
        b2 =  P(a0/a1)
        # compute a square-free factor...
        gj = P(b1/b2)
        # ...and append it to G
        if gj != 1 or not omit_trivial_factors :
            G.append( (gj, j) )
        # substitute bj by b_{j+1}
        b1 = b2
        # increment the counter
        j += 1
    return Factorization(G, unit = lcf, sort = false)

<h1>Exact roots of polynomials</h1>
<p><span style="color: #ff0000;">The above hidden cell will execute automatically upon opening the worksheet. It contains the following functions from exercise set 7:</span></p>
<ul>
<li><span style="color: #ff0000; font-family: courier new,courier;">is_square_free( f )</span></li>
<li><span style="color: #ff0000; font-family: courier new,courier;">rad( f )</span></li>
<li><span style="color: #ff0000; font-family: courier new,courier;">square_free_decomposition( f, omit_trivial_factors = true )</span></li>
</ul>
<p><span id="cell_outer_28"><span id="cell_outer_33"><strong>Exercise:</strong> Write a function that computes exact roots of a quadratic polynomial.</span></span></p>

In [8]:
def roots2( f ) :
    """
    This function computes the roots of a quadratic polynomial `f`
    """
    assert f.degree() == 2, "f must be a quadratic polynomial"
    
    # store the coefficients
    [c,b,a] = list(f)
    # the discriminant
    Delta = b^2-4*a*c
    # the roots
    return ( (-b-sqrt(Delta))/(2*a),  (-b+sqrt(Delta))/(2*a) )

In [10]:
# check it on 1000 random polynomials
P.<x> = QQ[]
for j in range(1000) :
    a = QQ.random_element()
    b = QQ.random_element()
    c = QQ.random_element()
    if c == 0 :
        continue
    f = c*(x-a)*(x-b)
    (x0, x1) = roots2(f)
    if f(x0) != 0 or f(x1) != 0 :
        print "Wrong result for ", f
        break
else :
    print "Passed"

Passed

<p><span id="cell_outer_2"><span id="cell_outer_28"><strong>Exercise:</strong> Write a function that computes all rational roots of a given polynomial using divisors of the free and constant coefficients.</span></span></p>

In [2]:
def rational_roots( f ) :
    """
    This function computes all the rational roots of a given polynomial `f` \
    with rational coefficients.
    """
    assert f.base_ring() <= QQ, "The ring of ceofficients must be contained in QQ."

    # find the radical of f
    fs = rad(f)
    # find the common denominator of the coefficients
    M = lcm([fi.denominator() for fi in fs])
    print "M=", M
    # ring of polys with integer coeffs
    P = ZZ[f.parent().0]
    # convert f to a polynomial with integer coefficients
    fz = P(M*fs)
    # find the primitive part
    fz = P( fz/gcd(list(fz)) )
    
    # if zero is a root of f...
    if fz(0) == 0 :
        # ...then append zero to the list of roots...
        C = [0]
        # ...and divide f by x
        # the following line just reads fz = fz/x, but
        # is safely independent of the actual name of the variable
        fz = P(fz/P.0)
    else:
        # otherwise we begin with an empty list of roots
        C = []
    # find the divizors of the constant term
    A = fz[0].divisors()
    # find the divisors of the leading coefficient
    B = fz.leading_coefficient().divisors()
    # check all the quotients...
    C += [ a/b for a in A for b in B if f(a/b) == 0 ]
    # ...don't forget about the negative ones
    C += [ -a/b for a in A for b in B if f(-a/b) == 0 ]
    # return the result
    return sorted(set(C))

In [5]:
# test it
P.<t> = QQ[]
f = (t-1)^3*(t-7)^2*(t+3/4)^2*(t-7/2)^3*(t^3-3*t+17); view("f = " + latex(f))
rational_roots(f)


M= 8
[-3/4, 1, 7/2, 7]

<p><span><strong>Exercise: </strong>Compute the roots of $f = 4 x^{3} - 40 x^{2} + 124 x - 120$</span>.</p>
<p>1) Define the polynomial $f$</p>

In [7]:
P.<x> = QQ[]
f = 4*x^3 - 40*x^2 + 124*x - 120; view("f = " + latex(f))

<p>2) Normalize the polynomial $f$ taking $f_1 \gets \frac{f}{\mbox{lc}(f)}$.</p>

In [14]:
f1 = P(f/f.leading_coefficient())
# alternative
#f1 = f.monic()
view("f_1 = " + latex(f1))

<p>3) Let $b_1$ denote the coefficient of $f_1$ at $x^2$. Depress the equation (i.e. translate the mean of the roots to the origin) substituting $f_2(x) := f_1(x - \frac{b_1}{3})$.</p>

In [12]:
b1 = f1[2]
f2 = f1( x = x - b1/3 ); view("f_2 = " + latex(f2))

<p>4) Denote the coefficient of $f_2$ at $x$ by $p$. Substitute $f_3 := x^3\cdot f_2(x-\frac{p}{3x})$.</p>

In [16]:
p = f2[1]
f3 = P( x^3* f2( x = x-p/(3*x) ) ); view("f_3 = " + latex(f3))

<p>5) Substitute $x^{1/3}$ for $x$ and convert the polynomial to a quadratic one (name it $f_4$).</p>
<p><em>Hint: Which elements of the lists of coefficients of $f_3$ are used to build $f_4$?</em></p>

In [18]:
f4 = P([ f3[j] for j in [0,3,6] ]); view("f_4 = " + latex(f4))

<p>6) Find the roots of $f_4$. Apply the previous function to it.</p>

In [20]:
T = roots2(f4); view(T)

<p>7) The roots (obtained in the previous step) are the roots <span style="text-decoration: underline;">the roots of $f_4$</span>. Revert all the substitutions to get the roots of the original polynomial $f$.</p>

In [22]:
t = T[1]
x0 = QQ(RR(t^(1/3)-p/(3*t^(1/3)) - b1/3)); view("x_0 = " + latex(x0))

<p>8) Divide $f$ by $x-x_0$, to get a quadratic polynomial. Compute its roots.</p>

In [24]:
g = f//(x-x0); view("g = " + latex(g))
(x1, x2) = roots2(g)
view("x_1 = " + latex(x1))
view("x_2 = " + latex(x2))

<p><strong>Exercise:</strong> Solve an equation $x^4 - 5x^2 + 6 = 0$.</p>
<p>1) Define the polynomial $f = x^4 - 5x^2 + 6$.</p>

In [26]:
P.<x> = QQ[]
f = x^4 - 5*x^2 + 6; f

x^4 - 5*x^2 + 6

<p>2) The polynomial $f$ is already depressed, denote its coefficients by $p,q,r$ so that $f = x^4 + px^2 + qx + r$.</p>

In [28]:
[r,q,p,_,_] = f.list()

<p>3) Construct a polynomial $h = -q^2 + \bigl(p^2 - 4 r\bigr) x + 2p x^2 + x^3$.</p>

In [31]:
h = -q^2 + (p^2-4*r)*x + 2*p*x^2 + x^3; view("h = " + latex(h))

<p>4) Find elements $\zeta_1, \dotsc, \zeta_3$ such that $h(\zeta_i^2) = 0$.</p>
<p><em>Hint:</em> Observe that one root is obvious.</p>

In [33]:
Zeta = [0] + list(roots2(h//x))
Zeta = [ sqrt(zeta) for zeta in Zeta ]
view("\\zeta\\in " + latex(Set(Zeta)) )

<p>5) Build a matrix \[ M = \frac{1}{2}\cdot \begin{pmatrix} 1 &  1 &  1 &  1\\ 1 & -1 &  1 & -1\\ 1 &  1 & -1 & -1\\ 1 & -1 & -1 &  1 \end{pmatrix} \]</p>

In [35]:
M = 1/2*matrix([[1,1,1,1],[1,-1,1,-1],[1,1,-1,-1],[1,-1,-1,1]])
view("M = " + latex(M))

<p>6) Check that it is its self-inverse, i.e. $M^{-1} = M$.</p>

In [37]:
view("M^2 = " + latex(M*M))

<p>7) Find the roots $\xi_0, \dotsc, \xi_3$ of the polynomial $f$ using the formula \[ \begin{pmatrix} \xi_0\\\xi_1\\\xi_2\\\xi_3\ \end{pmatrix} = M\cdot \begin{pmatrix} \zeta_0\\\zeta_1\\\zeta_2\\\zeta_3 \end{pmatrix} \]</p>

In [39]:
Xi = list(M*vector([0] + Zeta))
Xi = [ xi.simplify_full() for xi in Xi ]
view("\\xi\\in "+ latex(Set(Xi)))

<p>8) Verify numerically that $\xi_0, \dotsc, \xi_3$ are indeed roots of $f$.</p>

In [41]:
[ RR(xi) for xi in Xi ]

[1.73205080756888, -1.41421356237309, -1.73205080756888, 1.41421356237309]

In [43]:
[ f(RR(xi)) for xi in Xi ]

[1.77635683940025e-15,
 8.88178419700125e-16,
 1.77635683940025e-15,
 8.88178419700125e-16]